In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
MODEL_DIR = '../input/010-create-model-efficientnet-b0ns/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

## CFG

In [2]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    num_workers=4
    model_name='efficientnet_b0'
    size={'B0': 380, 'B0ns': 380, 
          'B1': 410, 'B1ns': 410, 
          'B2': 440, 'B2ns': 440, 
          'B3': 470, 'B3ns': 470, 
          'B4': 500, 'B4ns': 500}
    batch_size=128
    seed=42
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0]
    tta=3
    epochs = [7, 8, 9, 10]
    train=False
    inference=True


## Library

In [3]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Utils

In [4]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        print(f'Score: {score:<.5f}')

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')
    
def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

#LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

## Data Loading

In [5]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test.head()

,image_id,label
0,2216849948.jpg,4


## Dataset

In [6]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image


## Transforms

In [7]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            #Resize(CFG.size, CFG.size),
            RandomResizedCrop(CFG.size, CFG.size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'test':
        return Compose([
            RandomResizedCrop(CFG.size, CFG.size), 
#             Transpose(p=0.5), 
#             HorizontalFlip(p=0.5), 
#             VerticalFlip(p=0.5), 
#             ShiftScaleRotate(p=0.5), 
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
            
        ])

## MODEL

In [8]:
# Copyright (c) 2015-present, Facebook, Inc.
# All rights reserved.
import torch
import torch.nn as nn
from functools import partial

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master/timm')
from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.registry import register_model
from timm.models.layers import trunc_normal_

class DistilledVisionTransformer(VisionTransformer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dist_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        num_patches = self.patch_embed.num_patches
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 2, self.embed_dim))
        self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if self.num_classes > 0 else nn.Identity()

        trunc_normal_(self.dist_token, std=.02)
        trunc_normal_(self.pos_embed, std=.02)
        self.head_dist.apply(self._init_weights)

    def forward_features(self, x):
        # taken from https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
        # with slight modifications to add the dist_token
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        dist_token = self.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)

        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        return x[:, 0], x[:, 1]
    
    def forward(self, x):
        x, x_dist = self.forward_features(x)
        x = self.head(x)
        x_dist = self.head_dist(x_dist)
        if self.training:
            return x, x_dist
        else:
            # during inference, return the average of both classifier predictions
            return (x + x_dist) / 2
        
@register_model
def deit_base_patch16_384(pretrained=False, **kwargs):
    model = VisionTransformer(
        img_size=384, patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = _cfg()
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            url="https://dl.fbaipublicfiles.com/deit/deit_base_patch16_384-8de9b5d1.pth",
            map_location="cpu", check_hash=True
        )
        model.load_state_dict(checkpoint["model"])
    return model

In [9]:
# ====================================================
# MODEL
# ====================================================
class CustomEfficientNetB3ns(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x
    
class Custom_seresnext50_32x4d(nn.Module):
    def __init__(self, model_name='seresnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc= nn.Linear(n_features, CFG.target_size)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
class CustomDeiT(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=False):
        super().__init__()
        self.model = deit_base_patch16_384()
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

## Helper functions

In [10]:
# ====================================================
# Helper functions
# ====================================================
def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state['model'])
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

## inference

In [11]:
# ====================================================
# inference
# ====================================================
MODEL_DIR = '../input/nb023-data/'
CFG.model_name = 'tf_efficientnet_b3_ns'
CFG.size = 470
CFG.trn_fold=[0, 1, 2, 3, 4]

seed_torch(seed=CFG.seed)
model = CustomEfficientNetB3ns(CFG.model_name, pretrained=False)
# states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='test'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)

predictions_B3ns = []
for epoch in CFG.epochs:
    print(f'epoch: {epoch}')
    states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_epoch{epoch}.pth') for fold in CFG.trn_fold]
    each_epoch_predictions = []
    for tta in range(CFG.tta):
        predictions_ = inference(model, states, test_loader, device)
        each_epoch_predictions.append(predictions_)
    predictions_B3ns.append(np.mean(each_epoch_predictions, axis=0))
predictions_B3ns = np.mean(predictions_B3ns, axis=0)

# # submission
# test['label'] = predictions.argmax(1)
# test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
# test.head()

epoch: 7



epoch: 8



epoch: 9



epoch: 10


In [12]:
# ====================================================
# inference
# ====================================================
MODEL_DIR = '../input/nb028-data-2/'
CFG.model_name = 'seresnext50_32x4d'
CFG.size = 512
CFG.trn_fold=[0, 1, 2, 3, 4]

seed_torch(seed=CFG.seed)
model = Custom_seresnext50_32x4d(CFG.model_name, pretrained=False)
# states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='test'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)

predictions_se = []
for epoch in CFG.epochs:
    print(f'epoch: {epoch}')
    states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_epoch{epoch}.pth') for fold in CFG.trn_fold]
    each_epoch_predictions = []
    for tta in range(CFG.tta):
        predictions_ = inference(model, states, test_loader, device)
        each_epoch_predictions.append(predictions_)
    predictions_se.append(np.mean(each_epoch_predictions, axis=0))
predictions_se = np.mean(predictions_se, axis=0)

# # submission
# test['label'] = predictions.argmax(1)
# test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
# test.head()

epoch: 7



epoch: 8



epoch: 9



epoch: 10


In [13]:
# # ====================================================
# # inference
# # ====================================================
# MODEL_DIR = '../input/031-create-model-deit-data/'
# CFG.model_name = 'deit_base_patch16_384'
# CFG.size = 384
# CFG.trn_fold=[0]

# seed_torch(seed=CFG.seed)
# model = CustomDeiT(CFG.model_name, pretrained=False)
# # states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
# test_dataset = TestDataset(test, transform=get_transforms(data='test'))
# test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
#                          num_workers=CFG.num_workers, pin_memory=True)

# predictions_DeiT = []
# for epoch in CFG.epochs:
#     print(f'epoch: {epoch}')
#     states = [torch.load(MODEL_DIR+f'{CFG.model_name}_fold{fold}_epoch{epoch}.pth') for fold in CFG.trn_fold]
#     each_epoch_predictions = []
#     for tta in range(CFG.tta):
# #         print(f'tta_{tta}')
#         predictions_ = inference(model, states, test_loader, device)
#         each_epoch_predictions.append(predictions_)
#     predictions_DeiT.append(np.mean(each_epoch_predictions, axis=0))
# predictions_DeiT = np.mean(predictions_DeiT, axis=0)

# # # submission
# # test['label'] = predictions.argmax(1)
# # test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
# # test.head()

In [14]:
# submission
predictions = predictions_B3ns * 0.5 + predictions_se * 0.5
predictions[:, 4] *= 0.9 
test['label'] = predictions.argmax(1)
test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
test.head()

,image_id,label
0,2216849948.jpg,4
